# Algorithm 10: Triangle Multiplication Incoming (AlphaFold3)

Triangle Multiplication with incoming edges - symmetric to outgoing.

## Source Code Location
- **File**: `AF3-Ref-src/alphafold3-official/src/alphafold3/model/network/modules.py`

In [ ]:
import numpy as np
np.random.seed(42)

def layer_norm(x, eps=1e-5):
    mean = np.mean(x, axis=-1, keepdims=True)
    var = np.var(x, axis=-1, keepdims=True)
    return (x - mean) / np.sqrt(var + eps)

def sigmoid(x):
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

In [ ]:
def triangle_multiplication_incoming(z, c=128):
    """
    Triangle Multiplication with incoming edges.
    
    Updates z_ij using z_ki and z_kj for all k.
    
    Args:
        z: Pair representation [N, N, c_z]
        c: Hidden dimension
    
    Returns:
        Update to pair representation [N, N, c_z]
    """
    N = z.shape[0]
    c_z = z.shape[-1]
    
    print(f"Triangle Multiplication (Incoming)")
    print(f"="*50)
    print(f"Pair: [{N}, {N}, {c_z}]")
    
    z_norm = layer_norm(z)
    
    W_a = np.random.randn(c_z, c) * (c_z ** -0.5)
    W_ag = np.random.randn(c_z, c) * (c_z ** -0.5)
    W_b = np.random.randn(c_z, c) * (c_z ** -0.5)
    W_bg = np.random.randn(c_z, c) * (c_z ** -0.5)
    W_g = np.random.randn(c_z, c_z) * (c_z ** -0.5)
    
    a = sigmoid(z_norm @ W_ag) * (z_norm @ W_a)
    b = sigmoid(z_norm @ W_bg) * (z_norm @ W_b)
    g = sigmoid(z_norm @ W_g)
    
    # Triangle multiplication: sum_k a_ki * b_kj (incoming edges)
    # k ---> i, k ---> j
    output = np.einsum('kic,kjc->ijc', a, b)  # [N, N, c]
    
    print(f"Triangle product: {output.shape}")
    
    output = layer_norm(output)
    W_o = np.random.randn(c, c_z) * (c ** -0.5)
    output = (output @ W_o) * g
    
    print(f"Output: {output.shape}")
    
    return output

In [ ]:
# Test
print("Test: Triangle Multiplication Incoming")
print("="*60)

N = 32
c_z = 128

z = np.random.randn(N, N, c_z) * 0.1

output = triangle_multiplication_incoming(z, c=64)

print(f"\nOutput finite: {np.isfinite(output).all()}")

## Key Insights

1. **Incoming Edges**: Uses edges k→i and k→j to update i→j
2. **Complementary**: Works with outgoing to cover all triangle configurations
3. **Einsum Pattern**: `kic,kjc->ijc` vs `ikc,jkc->ijc` for outgoing